In [2]:
import sparknlp
import findspark
findspark.init()

spark = sparknlp.start()
print('spark version ' + spark.version)


22/10/07 09:56:29 WARN Utils: Your hostname, codespaces-a4c98f resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
22/10/07 09:56:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/root/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ba8aae72-2d2a-42ec-9f6c-c9d64c181804;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in 

spark version 3.2.1


In [3]:
from datetime import datetime, date
from pyspark.sql import *

from pyspark.sql.functions import *
from pyspark.ml import *
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.pretrained import *
from sparknlp.base import *

document = (
    DocumentAssembler()
    .setInputCol("text")
    .setOutputCol("document")
)

# we can also use sentence detector here
# if we want to train on and get predictions for each sentence
# downloading pretrained embeddings
sentence_encoder = (
    UniversalSentenceEncoder.pretrained()
    .setInputCols(["document"])
    .setOutputCol("sentence_embeddings")
)
# the classes/labels/categories are in category column
classsifierdl = (
    ClassifierDLApproach()
    .setInputCols(["sentence_embeddings"])
    .setLabelColumn("category")
    .setOutputCol("predicted_category")
    .setMaxEpochs(80)
    .setEnableOutputLogs(True)
)

# Training samples will look like this:
# `I look great in the mirror | [narcissistic, positive]`
# I do not think we want multi label predictions,
# this will do the same thing as the previous classifier
# but it will be taught on multiple labels and will be able to predict multiple labels.
# Although the potential is there, for example if we want to express more complicated emotions
# `How do I not get fired from my job? | [fear, anger, sadness]`
# multi_label_classifier = (
#     MultiClassifierDLApproach()
#     .setInputCols(["sentence_embeddings"])
#     .setLabelColumn("category")
#     .setOutputCol("predicted_category")
#     .setMaxEpochs(80)
#     .setEnableOutputLogs(True)
# )

pipeline = Pipeline(
    stages=[
        document,
        sentence_encoder,
        classsifierdl
    ]
)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
Download done! Loading the resource.
[ \ ]

2022-10-07 09:57:00.684560: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ | ]

2022-10-07 09:57:04.344562: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2022-10-07 09:57:04.437358: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2022-10-07 09:57:04.475311: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2022-10-07 09:57:04.511363: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2022-10-07 09:57:04.551332: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.


[OK!]


In [16]:
# Definitely feels like multi label would work much better.
# That's not even just for training, but also for predicting.
# What of something is both dumb and fun at the same time?
traing_samples = [
    # Work
    ['work', 'Work'],
    ['how to make a python3 spark dataframe?', 'Work'],
    ['i got tired after my presentation in front of colleagues', 'Work'],
    # Fun
    ['fun', 'Fun'],
    ['where to go out and eat with friends?', 'Fun'],
    ['Best things to bring to a beach party', 'Fun'],
    ['Fun things to do after a stressful day', 'Fun'],
    # Dumb
    # We actually do not want to label Dumb, as being dumb
    ['What does 2 + 2 equals', 'Dumb'],
    ['Is the earth flat?', 'Dumb'],
    # Smart
    ['How do I save for retirement?', 'Smart'],
    ['How do I stay healthy?', 'Smart'],
    # Violent
    ['Martial arts around me', 'Violent'],
    ['How do I beat somebody up?', 'Violent'],
    ['Murder videos', 'Violent'],
    # Illegal
    ['How do I break the law and get away with it?', 'Illegal'],
    ['How to I buy drugs?', 'Illegal'],
    # Medical
    ['My stomach hurts what do I do?', 'Medical'],
    ['I have a hard time focusing, do I have ADHD?', 'Medical'],
    # Sexual
    ['How do I use a condom?', 'Sexual'],
    ['Sex toys', 'Sexual'],
    ['How do I please my partner?', 'Sexual'],
    # Romantic
    ['Romantic movies', 'Romantic'],
    ['buy rose petals around me', 'Romantic'],
    # Incoherent
    ['adjf;qjdf asd fakldffalksdfj asd fasdkfljalkfd', 'Incoherent'],
    ['I am loooooeee aaaa confiss fond confused not a ssssentence why is this ap', 'Incoherent'],
]
training_data = spark.createDataFrame(traing_samples).toDF("text", "category")

all_labels = list(set(map(lambda x: x[1], traing_samples)))

fitted_pipeline = pipeline.fit(training_data)


2022-10-07 10:26:18.649345: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/cb3472373a9c_classifier_dl12996537496054888477
2022-10-07 10:26:18.720538: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-10-07 10:26:18.720583: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/cb3472373a9c_classifier_dl12996537496054888477
2022-10-07 10:26:19.035789: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-10-07 10:26:19.654604: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/cb3472373a9c_classifier_dl12996537496054888477
2022-10-07 10:26:19.777317: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 1127982 microse

Training started - epochs: 80 - learning_rate: 0.005 - batch_size: 64 - training_examples: 25 - classes: 10
Epoch 1/80 - 0.23s - loss: 2.3042173 - acc: 1.0 - batches: 1
Epoch 2/80 - 0.00s - loss: 2.2651052 - acc: 1.0 - batches: 1
Epoch 3/80 - 0.00s - loss: 2.1271913 - acc: 1.0 - batches: 1
Epoch 4/80 - 0.00s - loss: 1.901561 - acc: 1.0 - batches: 1
Epoch 5/80 - 0.00s - loss: 1.6891401 - acc: 1.0 - batches: 1
Epoch 6/80 - 0.00s - loss: 1.6008714 - acc: 1.0 - batches: 1
Epoch 7/80 - 0.00s - loss: 1.5774426 - acc: 1.0 - batches: 1
Epoch 8/80 - 0.00s - loss: 1.5614666 - acc: 1.0 - batches: 1
Epoch 9/80 - 0.00s - loss: 1.539887 - acc: 1.0 - batches: 1
Epoch 10/80 - 0.00s - loss: 1.5175974 - acc: 1.0 - batches: 1
Epoch 11/80 - 0.00s - loss: 1.5002327 - acc: 1.0 - batches: 1
Epoch 12/80 - 0.00s - loss: 1.4885907 - acc: 1.0 - batches: 1
Epoch 13/80 - 0.00s - loss: 1.4811559 - acc: 1.0 - batches: 1
Epoch 14/80 - 0.00s - loss: 1.4763964 - acc: 1.0 - batches: 1
Epoch 15/80 - 0.00s - loss: 1.47332

In [20]:
history = (
    spark.read.csv('sample_history.csv', sep='|', header=True)
    .toDF("kind", "text", "url")
)

labled = fitted_pipeline.transform(history)
labled.printSchema()


root
 |-- kind: string (nullable = true)
 |-- text: string (nullable = true)
 |-- url: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |  

In [ ]:

prediction = labled.predicted_category.getItem(0)

score_column_name = lambda category: f"{category}_score"

def category_score_column(category: str):
    return prediction.metadata.getItem(category).alias(score_column_name(category))

all_category_score_columns = [category_score_column(category) for category in all_labels]

print(f'Total history entries: {labled.count()}')

non_work_searches = (
    labled.select(
        [
            labled.text, 
            prediction.result.alias("predicted_category"),
        ]
        + all_category_score_columns
    )
    .filter(labled.kind == 'Searched')
    .filter(prediction.result != 'Work')
)

print(f'Non work related searches: {non_work_searches.count()}')


In [30]:

category_window_by_score = (
    Window.partitionBy(non_work_searches.predicted_category)
    .orderBy(non_work_searches.predicted_category + '_score')
)


cream_of_each_category = (
    non_work_searches.withColumn(
        "rank_within_category", 
        row_number().over(category_window_by_score)
    )
    .filter(col("rank_within_category") < 10)
)

cream_of_each_category.show(len(all_labels) * 10, False)


+--------------------------------------------------------------+------------------+------------+------------+--------------+-------------+-------------+----------------+------------+------------+-------------+-----------+--------------------+
|text                                                          |predicted_category|Dumb_score  |Sexual_score|Romantic_score|Medical_score|Violent_score|Incoherent_score|Fun_score   |Smart_score |Illegal_score|Work_score |rank_within_category|
+--------------------------------------------------------------+------------------+------------+------------+--------------+-------------+-------------+----------------+------------+------------+-------------+-----------+--------------------+
|modern skyscrapers earthquake codes magnitude                 |Dumb              |0.36136428  |0.015701747 |0.03664382    |0.14883885   |0.07628059   |0.052761618     |0.012273203 |0.02717181  |0.008050151  |0.26091394 |1                   |
|are la skyscrapers earthqua